In [50]:
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv
from urllib.parse import urlencode
import requests
import re

In [51]:
load_dotenv()
API_KEY = os.environ['SCRAPER_APIKEY']

In [52]:
session = requests.Session()

In [53]:
def get_scraperapi_url(search_term):

    def create_search_url(search_term):
        return 'https://www.amazon.com/s?' + urlencode({'k': search_term})
  
    payload = {'api_key': API_KEY, 'url': create_search_url(search_term)}
    proxy_url = 'http://api.scraperapi.com/?' + urlencode(payload)
    return proxy_url

In [54]:
def load_page(url):
    response = session.get(url)
    if response.status_code == 200:
        print(f'[{response.status_code}] OK')
        return response.content
    else:
        print(f'Failed to fetch the page. Status code: {response.status_code}')

In [55]:
get_price = lambda tag: tag.find('span', class_='a-offscreen').get_text(strip=True)

In [56]:
def get_discounts(soup):
    results = soup.find_all('div', attrs={'data-asin': True})

    discounts = []

    for div in results:
        if div['data-asin'] =='':
            continue
        prices = {}
            
        description_span = div.find('span', class_='a-size-medium a-color-base a-text-normal')

        deal_price_span = div.find('span', class_='a-price')
        if deal_price_span:
            prices['new_price'] = get_price(deal_price_span)
        
        old_price_span = div.find('span', class_='a-price a-text-price')
        if old_price_span:
            prices['old_price'] = get_price(old_price_span)
            discounts.append(prices)
            prices['ASIN'] = div['data-asin']
            prices['description'] = description_span.get_text(strip=True) if description_span else 'Not Found'
        else:
            continue

    return discounts

In [57]:
def next_page(link):
    page = 'https://www.amazon.com' + link  
    payload = {'api_key': API_KEY, 'url': page}
    proxy_url = 'http://api.scraperapi.com/?' + urlencode(payload)
    return proxy_url

In [58]:
def get_next_page_link(soup):
    for child in soup.find('span', class_='s-pagination-strip'):
        link = child.get('href')
        if link and child.text == "Next":
            return link

In [59]:
search = "Laptop"

In [60]:
url = get_scraperapi_url(search)
webpage = load_page(url)

[200] OK


In [61]:
if webpage:
    soup = BeautifulSoup(webpage, 'lxml')
    print(soup.title.text)    

Amazon.com : Laptop


In [62]:
discounts = get_discounts(soup)

print(len(discounts))

10


In [63]:
next = get_next_page_link(soup)
page_count = 1
while next:
    link = next_page(next)
    webpage = load_page(link)
    if webpage:
        page_count+=1
        _soup = BeautifulSoup(webpage, 'lxml')
        discounts.extend(get_discounts(_soup))
        next = get_next_page_link(_soup)

print(f"{page_count} pages {len(discounts)} Products Discovered for {search}")

Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to fetch the page. Status code: 503
Failed to f

KeyboardInterrupt: 